In [303]:
import math
from fractions import Fraction
from typing import List, Dict, Tuple, Optional
import collections 
from collections import Counter
import re
from pyzx.circuit import Circuit
from pyzx.circuit.gates import Gate, qasm_gate_table, ZPhase, XPhase, CRZ, gate_types

In [326]:
with open("C:\\Users\\Ronan Corless\\Documents\\GitHub\\ThesisRepo\\TestOPENQASMcode.txt", 'r') as file:
    data = file.read().replace('','')
with open("C:\\Users\\Ronan Corless\\Documents\\GitHub\\ThesisRepo\\TestcQASMcode.txt", 'r') as file:
    data2 = file.read().replace('','')
data2

'version 1.0;\nqubits 3; # setting three qubits\n# start writing your code here\nprep_z q[0,1,2];\nX q[0,1];\nZ q[2];\nH q[0];\nCNOT q[0],q[2]; # controlled not gate\nmeasure_all;'

In [305]:
lines = data2.splitlines()
r = []
#strip comments
for s in lines:
    if s.find("//")!=-1:
        t = s[0:s.find("//")].strip()
    else: t = s.strip()
    if t: r.append(t)

In [306]:
r
data2 = "\n".join(r)
data2

'version 1.0;\nqubits 3; # setting three qubits\n# start writing your code here\nprep_z q[0,1,2];\nX q[0,1];\nZ q[2];\nH q[0];\nCNOT q[0],q[2]; # controlled not gate\nmeasure_all;'

Replicating above but for cqasm

In [307]:
lines = data2.splitlines()
r = []
#strip comments
for s in lines:
    if s.find("#")!=-1:
        t = s[0:s.find("#")].strip()
    else: t = s.strip()
    if t: r.append(t)   
r.pop(0)

'version 1.0;'

In [308]:
data2 = "\n".join(r)
data2

'qubits 3;\nprep_z q[0,1,2];\nX q[0,1];\nZ q[2];\nH q[0];\nCNOT q[0],q[2];\nmeasure_all;'

In [309]:
commands = [s.strip() for s in data2.split(";") if s.strip()]
commands

['qubits 3',
 'prep_z q[0,1,2]',
 'X q[0,1]',
 'Z q[2]',
 'H q[0]',
 'CNOT q[0],q[2]',
 'measure_all']

In [310]:
parse_command('CNOT q[0],q[2]')

TypeError: parse_command() missing 2 required positional arguments: 'c' and 'registers'

In [315]:
class cQASMParser(object):
        "A class to parse cqasm txt files into zx circuit diagrams"
        def __init__(self)-> None: #initializing lists we will be using later
                self.gates: List[Gate] = []            
                self.registers: Dict[str,Tuple[int,int]] = {}
                self.qubit_count: int = 0
                self.circuit: Optional[Circuit] = None
    
        def parse(self, s: str, strict:bool=True) -> Circuit:
                s = s.lower()
                lines = s.splitlines() # spliting the input file by line
                r=[]
                for s in lines:
                        if s.find("#")!=-1: # clearing out the portion of any lines that comes after a hashtag ie. is commmented
                                t = s[0:s.find('#')].strip()
                        else: t = s.strip()
                        if t:r.append(t) # creating a list of the lines without comments
                if r[0].lower().startswith("version"):
                        r.pop(0) # if the first line starts with version, pop it out
                elif strict:
                        raise TypeError("File doesn't start with cQasm denotation") # denotes that wearent sure if it is a cqasm file
                data = "\n".join(r) # combines the list back into a text file with \n meaning new line
        
                commands = [s.strip() for s in data.split(";") if s.strip()]
                for c in commands:
                        self.gate.extend(self.parse_command(c, self.registers))
                circ = Circuit(self.qubit_count)
                circ.gates = self.gates
                self.circuit = circ
                return self.circuit
        
        
        def parse_command(commands):
                c = commands.split(' ')
        
                if len(c) == 1:
                        com = c
                elif len(c) == 2:
                        if c[1].count('[')== 1:
                                name = c[0]
                                rest = c[1]
                                name = name.lower()
                                rest = rest.lower()
                                if name in ["i","h","x","y","z","x90","y90","mx90","my90","s","sdag","t","tdag"]:
                                        val = rest.split('[',1)[1].split(']')[0]
                                        print("There is a ",name, " gate on the ", val, " qubit(s)")
                                if name.split('_')[0] in ["prep"]:
                                        val = name.split('_')[1]
                                        qubits = rest.split('[')[1].split(']')[0]
                                        print("The qubits ",qubits ,"are prepared in the", val,"basis.")
                                if name == "measure_all":
                                        return None
                                if name == 'qubits':
                                        self.qubit_count = rest
                        elif c[1].count('[') == 2:
                                name = c[0]
                                rest = c[1]
                                name = name.lower()
                                rest = rest.lower()
                                if name in ["cnot"]:
                                        val = rest.split(",")
                                        qubits = []
                                        for i in range(len(val)):
                                                qubit = val[i].split('[')[1].split(']')[0]
                                                qubits.append(int(qubit))
                                                qubits = sorted(qubits)
                                                print(qubits)
                                        print("There is a ",name,"gate on the qubits ", qubits)
    

In [316]:
attempt = cQASMParser()
attempt.parse(data2)

TypeError: File doesn't start with cQasm denotation

In [356]:
def parse_command(self, c:str, registers: Dict[str, Tuple[int,int]]) -> List[Gate]:
        gates: List[Gate] = []
        cbreak = c.split(' ')
        if len(cbreak) == 1:
                com = cbreak
        elif len(cbreak) == 2:
                if cbreak[1].count('[')== 1:
                        name = cbreak[0]
                        rest = cbreak[1]
                        name = name.lower()
                        rest = rest.lower()
                        if name in ["i","h","x","y","z","x90","y90","mx90","my90","s","sdag","t","tdag"]:
                                val = rest.split('[',1)[1].split(']')[0]
                                print("There is a ",name, " gate on the ", val, " qubit(s)")
                        if name.split('_')[0] in ["prep"]:
                                val = name.split('_')[1]
                                qubits = rest.split('[')[1].split(']')[0]
                                print("The qubits ",qubits ,"are prepared in the", val,"basis.")
                        if name == "measure_all":
                                return None
                        if name == 'qubits':
                                self.qubit_count = rest
                elif cbreak[1].count('[') == 2:
                        name = cbreak[0]
                        rest = cbreak[1]
                        name = name.lower()x
                        rest = rest.lower()
                        if name in ["cnot"]:
                                val = rest.split(",")
                                qubits = []
                                for i in range(len(val)):
                                        qubit = val[i].split('[')[1].split(']')[0]
                                        qubits.append(int(qubit))
                                        qubits = sorted(qubits)
                                        print(qubits)
                                print("There is a ",name,"gate on the qubits ", qubits)

SyntaxError: invalid syntax (Temp/ipykernel_17292/1084494676.py, line 26)

In [318]:
def parse_command(self, c: str, registers : Dict[str, Tuple[int,int]]) -> List[Gate]:
    gates: List[Gate] =[]
    name, rest = c.split(" ",1)
    print(rest)
    rest = ''.join(rest.split())
    qlist = []
    qlisttrue = []
    if rest.count("q[") != 0:
        for i in range(rest.count("q[")):
            rest2 = rest.split("q")
            rest2.pop(0)
        for j in range(len(rest2)):
            rest2[j] = rest2[j].split("[")[1]
            rest2[j] = rest2[j].split("]")[0]
            rest2[j] = rest2[j].split(",")
            for l in range(len(rest2[j])):
                if rest2[j][l].count(":") == 0:  
                    qlist.append(rest2[j][l])
                else:
                    firstside = (rest2[j][l].split(":")[0])[-1]
                    secondside = (rest2[j][l].split(":")[1])[0]
                    for k in range(int(firstside),int(secondside)+1):
                        qlist.append(k)
        qlist = [int(i) for i in qlist]
        for i in qlist:
            if i not in qlisttrue:
                qlisttrue.append(i)
        qlisttrue = qlisttrue.sort()
    if name == "qubits":
        self.qubit_count = rest
        #counts how many qubits we are working with
        return gates
    if name in ("i","h","x","y","z","x90","y90","mx90","my90","s","sdag","t","tdag"):
        #these are the non-argumentative single qubit gates
        onq = rest.split('[')[1].split(']')[0]
        howmanyq = onq.count(",")
        if howmanyq == (0):
            g = cqasm_gate_table[name]
    if name in ("x","z","s","t","h","sdag","tdag","h"): # these gates are already available as they exist in qasm_gate_table
        onq = rest.split('[')[1].split(']')[0]
        howmanyq = onq.count(",")
        if howmanyq == 0:
            g = cqasm_gate_table[name]
        if name in ('sdag'):
            g = qasm_gate_table["sdg"](argset[0],adjoint=True)
        elif name in ("tdag"):
            g = qasm_gate_table["sdg"](argset[0],adjoint=True)
        else:
            g = qasm_gate_table[name](argset[0])
        gates.append(g)
    if name in ("rx","ry","rz"):
        #single qubit gates with arguments for rotation
    if name in ("cr"):
        #single quibt gates with rotation around an arbitrary angle
    if name in ("crk"):
        #single quibt gates with rotation around a pi over 2^k gates
    if name in ("c-x","c-z"):
        #binary controlled rotation gates
    if name in ("cnot"):
        #2 qubit CNOT gates
    if name in ("toffoli"):
        #toffoli gates 
    if name in ("swap"):
        #swap gates on two qubits
    

IndentationError: expected an indented block (Temp/ipykernel_17292/665958171.py, line 53)

In [319]:
def parse_command(self, c: str, registers : Dict[str, Tuple[int,int]]) -> List[Gate]:
    gates: List[Gate] =[]
    name, rest = c.split(" ",1)
    if name == "qubits":
        self.qubit_count = rest

    if rest.count("q[") != 0:
        for i in range(rest.count("q[")):
            rest.split("q[")    
            content = rest.split('[')[1].split(']')[0]
            howmanygroup = content.count(",")
            qlist.append(howmanygroup) 

In [320]:
parse_command(c, d

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_17292/1509796833.py, line 1)

In [357]:
rest = "q[0,1,3:5,7] ,q[0,1,2:4],   q[0,1,2,9]"
rest = ''.join(rest.split())
qlist = []
qlisttrue = []
if rest.count("q[") != 0:
    for i in range(rest.count("q[")):
        rest2 = rest.split("q")
        rest2.pop(0)
    for j in range(len(rest2)):
        rest2[j] = rest2[j].split("[")[1]
        rest2[j] = rest2[j].split("]")[0]
        rest2[j] = rest2[j].split(",")
        for l in range(len(rest2[j])):
            if rest2[j][l].count(":") == 0:  
                qlist.append(rest2[j][l])
            else:
                firstside = (rest2[j][l].split(":")[0])[-1]
                secondside = (rest2[j][l].split(":")[1])[0]
                for k in range(int(firstside),int(secondside)+1):
                    qlist.append(k)
    qlist = [int(i) for i in qlist]
    for i in qlist:
        if i not in qlisttrue:
            qlisttrue.append(i)
    qlisttrue.sort()
    print(qlisttrue)

[0, 1, 2, 3, 4, 5, 7, 9]


In [354]:
class cQASMParser(object):
        "A class to parse cqasm txt files into zx circuit diagrams"
        def __init__(self)-> None: #initializing lists we will be using later
                self.gates: List[Gate] = []            
                self.registers: Dict[str,Tuple[int,int]] = {}
                self.qubit_count: int = 0
                self.circuit: Optional[Circuit] = None
    
        def parse(self, s: str, strict:bool=True) -> Circuit:
                s = s.lower()
                lines = s.splitlines() # spliting the input file by line
                r=[]
                for s in lines:
                        if s.find("#")!=-1: # clearing out the portion of any lines that comes after a hashtag ie. is commmented
                                t = s[0:s.find('#')].strip()
                        else: t = s.strip()
                        if t:r.append(t) # creating a list of the lines without comments
                if r[0].lower().startswith("version"):
                        r.pop(0) # if the first line starts with version, pop it out
                elif strict:
                        raise TypeError("File doesn't start with cQasm denotation") # denotes that wearent sure if it is a cqasm file
                data = "\n".join(r) # combines the list back into a text file with \n meaning new line
        
                commands = [s.strip() for s in data.split(";") if s.strip()]
                for c in commands:
                        self.gate.extend(self.parse_command(c, self.registers))
                circ = Circuit(self.qubit_count)
                circ.gates = self.gates
                self.circuit = circ
                return self.circuit
        
        
        def parse_commands(self, c:str, registers = Dict[str,Tuple[int,int]]) -> List[Gate]:
                gates: List[Gate] = []
                name, rest = c.split()
                rest = ''.join(rest.split())
                qlist = []
                qlisttrue = []
                if rest.count("q[") != 0:
                        for i in range(rest.count("q[")):
                                rest2 = rest.split("q")
                                rest2.pop(0)
                for j in range(len(rest2)):
                        rest2[j] = rest2[j].split("[")[1]
                        rest2[j] = rest2[j].split("]")[0]
                        rest2[j] = rest2[j].split(",")
                        for l in range(len(rest2[j])):
                                if rest2[j][l].count(":") == 0:  
                                        qlist.append(rest2[j][l])
                                else:
                                        firstside = (rest2[j][l].split(":")[0])[-1]
                                        secondside = (rest2[j][l].split(":")[1])[0]
                                for k in range(int(firstside),int(secondside)+1):
                                        qlist.append(k)
                qlist = [int(i) for i in qlist]
                for i in qlist:
                        if i not in qlisttrue:
                                qlisttrue.append(i)
                qlisttrue.sort()
                print(qlisttrue)
        


In [355]:
attempt = cQASMParser()
attempt.parse(data2)

AttributeError: 'cQASMParser' object has no attribute 'gate'

In [344]:
def parse_command4(c):
    gates: List[Gate] = []
    name, rest = c.split(" ",1)
    if name in ("barrier","creg","measure", "id"): return gates
    if name in ("opaque", "if"):
        raise TypeError("Unsupported operation {}".format(c))
    args = [s.strip() for s in rest.split(",") if s.strip()]
    print(args)


In [346]:
parse_command4(data)

['2.0;\ninclude "qelib1.inc";\nqreg a[3];\t\ncreg meas[3];\nx a[0]; // here I am creating a gate\nx a[1];\nz a[2];\nh a[0];\ncx a[0]', 'a[2];\nbarrier a[0]', 'a[1]', 'a[2]; // oh look a gate\nmeasure a[0] -> meas[0];\nmeasure a[1] -> meas[1];\nmeasure a[2] -> meas[2]; // wow', 'measurements!']
